In [3]:
import pandas as pd
import numpy as np
import datetime
from sim import generate_simulation
from model_cancellations import get_otb_res, predict_cancellations
import re

pd.options.display.max_rows = 150
pd.options.display.max_columns = 150

In [4]:
DATE_FMT = "%Y-%m-%d"
h1_capacity = 187
h2_capacity = 226
AOD = "2017-08-01"
AOD_dt = pd.to_datetime(AOD)

h1_res = pd.read_pickle("pickle/h1_res.pick")
h2_res = pd.read_pickle("pickle/h2_res.pick")
h1_dbd = pd.read_pickle("pickle/h1_dbd.pick")
h2_dbd = pd.read_pickle("pickle/h2_dbd.pick")
h1_sim = pd.read_pickle("pickle/h1_sim.pick")
h2_sim = pd.read_pickle("pickle/h2_sim.pick")

In [6]:
h1_sim["TM05_Date"] = h1_sim.Date - pd.DateOffset(5)
h1_sim["TM15_Date"] = h1_sim.Date - pd.DateOffset(15)
h1_sim["TM30_Date"] = h1_sim.Date - pd.DateOffset(30)

In [7]:
h1_sim.head(3)

,DOW,RoomsOTB,RevOTB,CxlForecast,Trn_RoomsOTB,Trn_RevOTB,Trn_CxlForecast,TrnP_RoomsOTB,TrnP_RevOTB,TrnP_CxlForecast,Grp_RoomsOTB,Grp_RevOTB,Grp_CxlForecast,Cnt_RoomsOTB,Cnt_RevOTB,Cnt_CxlForecast,Date,WeekEndDate,WE,WD,STLY_Date,Occ,RevPAR,RemSupply,ADR_OTB,Trn_ADR_OTB,TrnP_ADR_OTB,Grp_ADR_OTB,Cnt_ADR_OTB,LYA_RoomsSold,LYA_ADR,LYA_RoomRev,LYA_RevPAR,LYA_NumCancels,SellingPrice,tm05_RoomsOTB,tm05_ADR,tm05_SellingPrice,tm05_TRN_OTB,tm05_TRN_ADR,tm05_TRNP_OTB,tm05_TRNP_ADR,tm05_GRP_OTB,tm05_GRP_ADR,tm05_CNT_OTB,tm05_CNT_ADR,tm15_RoomsOTB,tm15_ADR,tm15_SellingPrice,tm15_TRN_OTB,tm15_TRN_ADR,tm15_TRNP_OTB,tm15_TRNP_ADR,tm15_GRP_OTB,tm15_GRP_ADR,tm15_CNT_OTB,tm15_CNT_ADR,tm30_RoomsOTB,tm30_ADR,tm30_SellingPrice,tm30_TRN_OTB,tm30_TRN_ADR,tm30_TRNP_OTB,tm30_TRNP_ADR,tm30_GRP_OTB,tm30_GRP_ADR,tm30_CNT_OTB,tm30_CNT_ADR,STLY_OTB,STLY_Rev,STLY_ADR,STLY_SellingPrice,STLY_TRN_OTB,STLY_TRN_REV,STLY_TRN_ADR,STLY_TRNP_OTB,STLY_TRNP_REV,STLY_TRNP_ADR,STLY_GRP_OTB,STLY_GRP_REV,STLY_GRP_ADR,STLY_CNT_OTB,STLY_CNT_REV,STLY_CNT_ADR,STLY_TM05_OTB,STLY_TM05_ADR,STLY_TM15_OTB,STLY_TM15_ADR,STLY_TM30_OTB,STLY_TM30_ADR,STLY_TM05_TRN_OTB,STLY_TM05_TRN_ADR,STLY_TM15_TRN_OTB,STLY_TM15_TRN_ADR,STLY_TM30_TRN_OTB,STLY_TM30_TRN_ADR,STLY_TM05_TRNP_OTB,STLY_TM05_TRNP_ADR,STLY_TM15_TRNP_OTB,STLY_TM15_TRNP_ADR,STLY_TM30_TRNP_OTB,STLY_TM30_TRNP_ADR,STLY_TM05_GRP_OTB,STLY_TM05_GRP_ADR,STLY_TM15_GRP_OTB,STLY_TM15_GRP_ADR,STLY_TM30_GRP_OTB,STLY_TM30_GRP_ADR,STLY_TM05_CNT_OTB,STLY_TM05_CNT_ADR,STLY_TM15_CNT_OTB,STLY_TM15_CNT_ADR,STLY_TM30_CNT_OTB,STLY_TM30_CNT_ADR,TM05_Date
2017-08-01,Tue,184.0,35239.14,18.0,151.0,29935.72,16.0,13.0,2488.24,2.0,2.0,291.66,0.0,18.0,2523.52,0.0,2017-08-01,2017-08-06,False,False,2016-08-02,0.98,188.44,21,191.52,198.25,191.40,145.83,140.20,183.0,170.00,31110.89,166.37,81.0,209.01,184.0,191.52,206.48,151.0,198.25,13.0,191.40,2.0,145.83,18.0,140.20,184.0,191.52,206.48,151.0,198.25,13.0,191.40,2.0,145.83,18.0,140.20,184.0,191.52,197.59,151.0,198.25,13.0,191.40,2.0,145.83,18.0,140.20,183.0,31110.89,170.00,185.03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,183.0,170.00,183.0,170.00,183.0,170.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017-07-27
2017-08-02,Wed,186.0,36141.47,19.0,152.0,30940.93,17.0,11.0,2004.24,2.0,2.0,291.66,0.0,21.0,2904.64,0.0,2017-08-02,2017-08-06,False,False,2016-08-03,0.99,193.27,20,194.31,203.56,182.20,145.83,138.32,182.0,173.96,31660.62,169.31,86.0,209.01,186.0,194.31,206.48,152.0,203.56,11.0,182.20,2.0,145.83,21.0,138.32,186.0,194.31,206.48,152.0,203.56,11.0,182.20,2.0,145.83,21.0,138.32,186.0,194.31,201.27,152.0,203.56,11.0,182.20,2.0,145.83,21.0,138.32,182.0,31660.62,173.96,187.13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,182.0,173.96,182.0,173.96,182.0,173.96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017-07-28
2017-08-03,Thu,179.0,35705.54,17.0,140.0,29773.21,15.0,13.0,2225.28,2.0,3.0,453.06,0.0,23.0,3253.99,0.0,2017-08-03,2017-08-06,False,False,2016-08-04,0.96,190.94,25,199.47,212.67,171.18,151.02,141.48,183.0,182.11,33326.17,178.21,96.0,209.01,179.0,199.47,206.48,140.0,212.67,13.0,171.18,3.0,151.02,23.0,141.48,179.0,199.47,206.48,140.0,212.67,13.0,171.18,3.0,151.02,23.0,141.48,179.0,199.47,203.79,140.0,212.67,13.0,171.18,3.0,151.02,23.0,141.48,183.0,33326.17,182.11,189.50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,183.0,182.11,183.0,182.11,183.0,182.11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017-07-29


In [3]:
# h1_stly_sim = generate_simulation(h1_dbd, "2016-08-02", 1, h1_res)

In [ ]:
h1_tm_otb_cols = [col for col in h1_sim.columns if re.search(r"[Tt]?[Mm]?[0-9]{1,2}_OTB", col) is not None]
# h1_tm_stly_cols = [col for col in h1_sim.columns if re.search(r"STLY.*[Tt][Mm][0-9]{1,2}_OTB", col) is not None]
# h1_tm_cols = h1_tm_otb_cols + h1_tm_stly_cols
h1_tm_otb_cols

## Add Price Columns

In [11]:
night = '2017-08-12'
h1_pricing = h1_res.copy()
mask = (h1_pricing.ArrivalDate <=night) & (h1_pricing.CheckoutDate > night) & (h1_pricing.CustomerType == 'Transient')
price = h1_pricing[mask].ADR.mean()

214.90617021276594

## Add Pace columns



In [ ]:
h2_sim[h2_tm_cols].loc["2017-08-20"]

In [5]:
STLY_COLS = [col for col in h1_sim.columns if col[:5] == 'STLY_']
LYA_COLS = [col for col in h1_sim.columns if col[:4] == 'LYA_']

lam = lambda x: x not in STLY_COLS+LYA_COLS and x[-3:] == 'OTB'
OTB_COLS = [col for col in h1_sim.columns if lam(col)]

In [ ]:
h1_sim[h2_tm_otb_cols].loc["2017-08-02"]

In [6]:
OTB_COLS

['RoomsOTB',
 'RevOTB',
 'Trn_RoomsOTB',
 'Trn_RevOTB',
 'TrnP_RoomsOTB',
 'TrnP_RevOTB',
 'Grp_RoomsOTB',
 'Grp_RevOTB',
 'Cnt_RoomsOTB',
 'Cnt_RevOTB',
 'ADR_OTB',
 'Trn_ADR_OTB',
 'TrnP_ADR_OTB',
 'Grp_ADR_OTB',
 'Cnt_ADR_OTB',
 'tm05_RoomsOTB',
 'tm05_TRN_OTB',
 'tm05_TRNP_OTB',
 'tm05_GRP_OTB',
 'tm05_CNT_OTB',
 'tm15_RoomsOTB',
 'tm15_TRN_OTB',
 'tm15_TRNP_OTB',
 'tm15_GRP_OTB',
 'tm15_CNT_OTB',
 'tm30_RoomsOTB',
 'tm30_TRN_OTB',
 'tm30_TRNP_OTB',
 'tm30_GRP_OTB',
 'tm30_CNT_OTB']

In [7]:
STLY_COLS

['STLY_Date',
 'STLY_OTB',
 'STLY_Rev',
 'STLY_ADR',
 'STLY_SellingPrice',
 'STLY_TRN_OTB',
 'STLY_TRN_REV',
 'STLY_TRN_ADR',
 'STLY_TRNP_OTB',
 'STLY_TRNP_REV',
 'STLY_TRNP_ADR',
 'STLY_GRP_OTB',
 'STLY_GRP_REV',
 'STLY_GRP_ADR',
 'STLY_CNT_OTB',
 'STLY_CNT_REV',
 'STLY_CNT_ADR',
 'STLY_TM05_OTB',
 'STLY_TM05_ADR',
 'STLY_TM15_OTB',
 'STLY_TM15_ADR',
 'STLY_TM30_OTB',
 'STLY_TM30_ADR',
 'STLY_TM05_TRN_OTB',
 'STLY_TM05_TRN_ADR',
 'STLY_TM15_TRN_OTB',
 'STLY_TM15_TRN_ADR',
 'STLY_TM30_TRN_OTB',
 'STLY_TM30_TRN_ADR',
 'STLY_TM05_TRNP_OTB',
 'STLY_TM05_TRNP_ADR',
 'STLY_TM15_TRNP_OTB',
 'STLY_TM15_TRNP_ADR',
 'STLY_TM30_TRNP_OTB',
 'STLY_TM30_TRNP_ADR',
 'STLY_TM05_GRP_OTB',
 'STLY_TM05_GRP_ADR',
 'STLY_TM15_GRP_OTB',
 'STLY_TM15_GRP_ADR',
 'STLY_TM30_GRP_OTB',
 'STLY_TM30_GRP_ADR',
 'STLY_TM05_CNT_OTB',
 'STLY_TM05_CNT_ADR',
 'STLY_TM15_CNT_OTB',
 'STLY_TM15_CNT_ADR',
 'STLY_TM30_CNT_OTB',
 'STLY_TM30_CNT_ADR']

In [8]:
LYA_COLS

['LYA_RoomsSold', 'LYA_ADR', 'LYA_RoomRev', 'LYA_RevPAR', 'LYA_NumCancels']

In [9]:
h1_sim["RoomsPace"] = h1_sim.RoomsOTB - h1_sim.STLY_OTB
h1_sim["RoomsPace"] = h1_sim.RoomsOTB - h1_sim.STLY_OTB

## Add T - 30 OTB Features

This will allow us to compare recent pickup vs. STLY